In [ ]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("../data_base/knowledge_path/pumkin_book/pumpkin_book.pdf")
pages = loader.load()
print(pages)

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://zh.d2l.ai/")
docs = loader.load()
print(docs[0].page_content[:500])

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
some_text = """When writing documents, writers will use document structure to group content. \
This can convey to the reader, which idea's are related. For example, closely related ideas \
are in sentances. Similar ideas are in paragraphs. Paragraphs form a document. \n\n  \
Paragraphs are often delimited with a carriage return or two carriage returns. \
Carriage returns are the "backslash n" you see embedded in this string. \
Sentences have a period at the end, but also, have a space.\
and words are separated by space."""

c_splitter = CharacterTextSplitter(
    chunk_size=450,
    chunk_overlap=0,
    separator = ' '
)
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=450,
    chunk_overlap=0, 
    separators=["\n\n", "\n", " ", ""])

In [ ]:
print(c_splitter.split_text(some_text))

In [ ]:
print(r_splitter.split_text(some_text))

In [ ]:
# https://python.langchain.com/api_reference/community/embeddings/langchain_community.embeddings.zhipuai.ZhipuAIEmbeddings.html#langchain_community.embeddings.zhipuai.ZhipuAIEmbeddings

from langchain_community.embeddings import ZhipuAIEmbeddings

embed = ZhipuAIEmbeddings(
    model="embedding-2",
    api_key="5713143e8fdc4b4a8b284cf97092e70f.qEK71mGIlavzO1Io",
)
input_text = "The meaning of life is 42"
embed.embed_query(input_text)[:10]

In [ ]:
from langchain_chroma import Chroma
# 存放文件路径设置
persist_directory = r'./langchain'

# 创建向量数据库
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
)
print(vectordb._collection.count())

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import ZhipuAIEmbeddings
from langchain_chroma import Chroma
# 文件导入
loader = WebBaseLoader("https://zh.d2l.ai/")
docs = loader.load()

# 文本切分
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)
splits = text_splitter.split_documents(docs)
print(len(splits))

# 文本嵌入
embed = ZhipuAIEmbeddings(
    model="embedding-2",
    api_key="5713143e8fdc4b4a8b284cf97092e70f.qEK71mGIlavzO1Io",
)

# 测试
# text_1 = "今天天气不错"

# query_result = embeddings.embed_query(text_1)
# print(query_result)

# 路径设置
persist_directory = r'./langchain-chroma'

# 向量库创建
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embed,
    persist_directory=persist_directory
)
print(vectordb._collection.count())

# 检索
question = "图像识别"
docs = vectordb.similarity_search(question,k=3)
print(len(docs))
print(docs[0].page_content)

In [ ]:
# 假设你有一个 vectordb 对象，并且可以获取所有文档
all_docs = vectordb.similarity_search("", k=18)  # 可以使用一个空查询或任意查询返回多个文档

# 遍历文档并输出每个文档的元数据
for i, doc in enumerate(all_docs):
    print(f"Document {i+1} metadata: {doc.metadata}")

In [ ]:
# 添加新文档
new_loader = WebBaseLoader("https://www.deeplearning.ai/")  # 新文档来源
new_docs = new_loader.load()

# 文本切分
new_splits = text_splitter.split_documents(new_docs)

# 添加到现有的向量库
vectordb.add_documents(new_splits)

# 输出更新后的文档数量
print(f"更新后的文档数量: {vectordb._collection.count()}")

In [ ]:
# 假设你有一个 vectordb 对象，并且可以获取所有文档
all_docs = vectordb.similarity_search("", k=18)  # 可以使用一个空查询或任意查询返回多个文档

# 遍历文档并输出每个文档的元数据
for i, doc in enumerate(all_docs):
    print(f"Document {i+1} metadata: {doc.metadata}")

In [ ]:
# 假设你有一个 vectordb 对象，并且可以获取所有文档
all_docs = vectordb.similarity_search("", k=18)  # 可以使用一个空查询或任意查询返回多个文档

# 遍历文档并输出每个文档的元数据
for i, doc in enumerate(all_docs):
    print(f"Document {i+1} metadata: {doc.metadata}")

In [ ]:
question = "大语言模型是怎么推理的？"
docs_meta = vectordb.similarity_search(question, k=3, filter={"source": "https://zh.d2l.ai/"})
print(docs_meta[0].page_content[:])

In [ ]:
from langchain_community.chat_models import ChatZhipuAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_community.embeddings import BaichuanTextEmbeddings
from langchain_chroma import Chroma  # 从 langchain_chroma 中引用 Chroma 类
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 文件导入
loader = WebBaseLoader("https://zh.d2l.ai/")
docs = loader.load()

# 文本切分
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)
splits = text_splitter.split_documents(docs)

persist_directory = './langchain-chroma'

# 初始化嵌入
embed = ZhipuAIEmbeddings(
    model="embedding-2",
    api_key="5713143e8fdc4b4a8b284cf97092e70f.qEK71mGIlavzO1Io",
)

# 使用 embedding_function 参数初始化 Chroma
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embed
)

chat = ChatZhipuAI(
    model="glm-4-flash",
    temperature=0.5,
    api_key =  "5713143e8fdc4b4a8b284cf97092e70f.qEK71mGIlavzO1Io"
)

from langchain.chains import RetrievalQA
question = "这本书最重要的主题是?"
qa_chain = RetrievalQA.from_chain_type(
    chat, # 大模型的选用
    retriever=vectordb.as_retriever() # 向量数据库的调用
)
result = qa_chain({"query": question}) # 输入问题
print(result["result"]) # 获取回复